In [6]:
import math
from openpyxl import load_workbook
from unipath import Path
import os
from django.core import exceptions

base = Path('d:\\', 'Users', 'javier', 'tmp')
r06 = Path(base, 'mac_06.xlsx')
r07 = Path(base, 'mac_07.xlsx')
r07b = Path(base, 'mac_07b.xlsx')
# wb6 = load_workbook(filename = r06)
# wb7 = load_workbook(filename = r07)
# wb7b = load_workbook(filename = r07b)
# ws6 = wb6['2021_06']
# ws7 = wb7['2021_06']
# ws7b = wb7b['2021_06']

In [7]:
def get_int(celda):
    """Convierte el valor de una celda en entero"""
    if celda.value == '':
        return 0
    try:
        if celda.ctype != 2:
            return 0
        return math.ceil(celda.value)
    except ValueError:
        return 0


def get_float(celda):
    """"Convierte el valor de una celda en un número flotante"""
    # valor = 0
    try:
        valor = float(celda.value)
    except ValueError:
        valor = 0
    return valor

In [14]:
def get_data_excel(file):
    try: 
        wb = load_workbook(filename = file, data_only=True)
    except Exception: 
        raise BadRequest(e)
    ws = wb.active
    remesa = ws['J7'].value[8:15]
    observaciones = ws['A41'].value
    macs = {}
    
    for row in range(9, 22):
        mac = ws[row]
        try:
            _mac = str(mac[0].value)
            if _mac[:3] == '290':
                macs[_mac] = {
                    'distrito': _mac[2:4],
                    'type': mac[1].value,
                    'days_worked': mac[2].value,
                    'hours_worked': mac[3].value,
                    'setup': mac[4].value,
                    'tramites_aplicados': mac[5].value,
                    'cards_by_update': mac[6].value,
                    'total_atenciones': mac[7].value,
                    'numbers_by_day': mac[8].value,
                    'numbers_by_machine_by_day': mac[9].value,
                    'credenciales_recibidas': mac[11].value
                }
        except ValueError:
            pass
    
    return remesa, observaciones, macs

r, o, macs = get_data_excel(r06)

In [17]:
print(macs)

{'290151': {'distrito': '01', 'type': 'Fijo Dist.', 'days_worked': 6.5, 'hours_worked': 10.2, 'setup': 'B+3', 'tramites_aplicados': 1482, 'cards_by_update': 17, 'total_atenciones': 1499, 'numbers_by_day': 230.6153846153846, 'numbers_by_machine_by_day': 57.65384615384615, 'credenciales_recibidas': 1304}, '290152': {'distrito': '01', 'type': 'Fijo Adic.', 'days_worked': 6.5, 'hours_worked': 10.2, 'setup': 'B+2', 'tramites_aplicados': 1282, 'cards_by_update': 68, 'total_atenciones': 1350, 'numbers_by_day': 207.69230769230768, 'numbers_by_machine_by_day': 69.23076923076923, 'credenciales_recibidas': 357}, '290153': {'distrito': '01', 'type': 'Móvil', 'days_worked': 6.5, 'hours_worked': 6.5, 'setup': 'B', 'tramites_aplicados': 330, 'cards_by_update': 21, 'total_atenciones': 351, 'numbers_by_day': 54, 'numbers_by_machine_by_day': 54, 'credenciales_recibidas': 339}, '290154': {'distrito': '01', 'type': 'Móvil', 'days_worked': 6.5, 'hours_worked': 6.5, 'setup': 'B', 'tramites_aplicados': 211, 